## Data Processing

The data processing procedure outlined aims to handle missing data and outliers for datasets related to wind farms and solar stations, each with a specified nominal capacity. The process ensures the integrity and usability of the data for subsequent analysis or predictive modeling tasks. Each dataset undergoes systematic treatment to identify and rectify missing values, followed by the detection and handling of outliers, which could distort subsequent analyses.

#### Data Cleaning Steps:
1. Load the Datasets:

Load data from multiple Excel files for wind farms and solar stations, dropping the first row in each file which presumably contains template or incorrect data.

2. Define Cleaning Function (clean_and_handle_outliers):

Replace Placeholder Values: Substitute placeholder and incorrect values ('0.001', '−99', '–', 'NA', '--', '<NULL>', '') with NaN (Not a Number).
Correct Wind Speed Values: Ensure that wind speed values are non-negative by replacing any negative values with NaN.
Handle Temperature Values: Filter out unreasonable temperature readings (less than -100°C or greater than 100°C) by replacing them with NaN.
Interpolate Numeric Columns: Perform linear interpolation on numeric columns to fill in missing data, applying both forward and backward limits.
Fill Remaining Missing Data: Use forward fill and backward fill methods to handle any remaining missing values.
Handle Outliers: Identify outliers using the Interquartile Range (IQR) method and cap them at 1.5 times the IQR above the third quartile and below the first quartile.

3. Apply the Cleaning Function:

Process each dataset individually based on whether they are wind farm (WF) or solar station (SS) datasets, using appropriate columns for wind speed and temperature.
Output Processed Datasets:

4. Save each cleaned and processed dataset back into an Excel file within a specified directory for processed data.

In [54]:
import pandas as pd
import numpy as np
    
# Load datasets
datasets = {
    'WF_site1': pd.read_excel('../renewable_energy_data/data_original/wind_farms/Wind farm site 1 (Nominal capacity-99MW).xlsx').drop(index=0),
    'WF_site2': pd.read_excel('../renewable_energy_data/data_original/wind_farms/Wind farm site 2 (Nominal capacity-200MW).xlsx').drop(index=0),
    'WF_site3': pd.read_excel('../renewable_energy_data/data_original/wind_farms/Wind farm site 3 (Nominal capacity-99MW).xlsx').drop(index=0),
    'WF_site4': pd.read_excel('../renewable_energy_data/data_original/wind_farms/Wind farm site 4 (Nominal capacity-66MW).xlsx').drop(index=0),
    'WF_site5': pd.read_excel('../renewable_energy_data/data_original/wind_farms/Wind farm site 5 (Nominal capacity-36MW).xlsx').drop(index=0),
    'WF_site6': pd.read_excel('../renewable_energy_data/data_original/wind_farms/Wind farm site 6 (Nominal capacity-96MW).xlsx').drop(index=0),
    'SS_site1': pd.read_excel('../renewable_energy_data/data_original/solar_stations/Solar station site 1 (Nominal capacity-50MW).xlsx').drop(index=0),
    'SS_site2': pd.read_excel('../renewable_energy_data/data_original/solar_stations/Solar station site 2 (Nominal capacity-130MW).xlsx').drop(index=0),
    'SS_site3': pd.read_excel('../renewable_energy_data/data_original/solar_stations/Solar station site 3 (Nominal capacity-30MW).xlsx').drop(index=0),
    'SS_site4': pd.read_excel('../renewable_energy_data/data_original/solar_stations/Solar station site 4 (Nominal capacity-130MW).xlsx').drop(index=0),
    'SS_site5': pd.read_excel('../renewable_energy_data/data_original/solar_stations/Solar station site 5 (Nominal capacity-110MW).xlsx').drop(index=0),
    'SS_site6': pd.read_excel('../renewable_energy_data/data_original/solar_stations/Solar station site 6 (Nominal capacity-35MW).xlsx').drop(index=0),
    'SS_site7': pd.read_excel('../renewable_energy_data/data_original/solar_stations/Solar station site 7 (Nominal capacity-30MW).xlsx').drop(index=0),
    'SS_site8': pd.read_excel('../renewable_energy_data/data_original/solar_stations/Solar station site 8 (Nominal capacity-30MW).xlsx').drop(index=0)
}





In [55]:
def clean_and_handle_outliers(df, wind_speed_cols, temp_col):
    # Replace placeholder and incorrect values with NaN
    placeholders = ['0.001', '−99', '–', 'NA', '--', '<NULL>', '']
    df.replace(placeholders, np.nan, inplace=True)

    # Correct negative wind speed values
    for col in wind_speed_cols:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: np.nan if x < 0 else x)

    # Handle unreasonable air temperature values
    if temp_col in df.columns:
        df[temp_col] = df[temp_col].apply(lambda x: np.nan if x < -100 or x > 100 else x)

    # Interpolate missing data for numerical columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].interpolate(method='linear', limit_direction='both')

    # Forward fill and backward fill remaining missing data
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)

    # Adjust outlier handling to ignore zero values that are placeholders
    # and not convert valid non-zero readings to NaNs
    for col in numeric_cols:
        if col in wind_speed_cols:
            # Only apply outlier capping to non-zero values
            non_zero_values = df[col] != 0
            Q1 = df.loc[non_zero_values, col].quantile(0.25)
            Q3 = df.loc[non_zero_values, col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Cap outliers
            df.loc[non_zero_values, col] = np.where(df.loc[non_zero_values, col] < lower_bound, lower_bound, df.loc[non_zero_values, col])
            df.loc[non_zero_values, col] = np.where(df.loc[non_zero_values, col] > upper_bound, upper_bound, df.loc[non_zero_values, col])
        else:
            # Handle outliers using the interquartile range for other numerical columns
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Cap outliers
            df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
            df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])

    return df
# Now apply the cleaning and outlier handling function to each dataset
processed_datasets = {}
for key, dataset in datasets.items():
    print(f"Processing {key}...")
    if 'WF' in key:  # For Wind farm datasets
        processed_datasets[key] = clean_and_handle_outliers(dataset, wind_farm_cols, temperature_col)
    elif 'SS' in key:  # For Solar station datasets
        processed_datasets[key] = clean_and_handle_outliers(dataset, solar_station_cols, temperature_col)
    else:
        # Apply a general cleaning if the dataset key doesn't specify WF or SS
        processed_datasets[key] = clean_and_handle_outliers(dataset, wind_farm_cols + solar_station_cols, temperature_col)
    print(f"Finished processing {key}. Shape: {processed_datasets[key].shape}")

Processing WF_site1...
Finished processing WF_site1. Shape: (70175, 13)
Processing WF_site2...
Finished processing WF_site2. Shape: (70175, 13)
Processing WF_site3...
Finished processing WF_site3. Shape: (70175, 13)
Processing WF_site4...
Finished processing WF_site4. Shape: (70175, 13)
Processing WF_site5...
Finished processing WF_site5. Shape: (70175, 13)
Processing WF_site6...
Finished processing WF_site6. Shape: (70175, 13)
Processing SS_site1...
Finished processing SS_site1. Shape: (70175, 8)
Processing SS_site2...
Finished processing SS_site2. Shape: (70175, 8)
Processing SS_site3...
Finished processing SS_site3. Shape: (52607, 8)
Processing SS_site4...
Finished processing SS_site4. Shape: (70175, 8)
Processing SS_site5...
Finished processing SS_site5. Shape: (70175, 8)
Processing SS_site6...
Finished processing SS_site6. Shape: (70175, 8)
Processing SS_site7...
Finished processing SS_site7. Shape: (70175, 8)
Processing SS_site8...
Finished processing SS_site8. Shape: (69407, 8)


In [56]:
import os

# Specify the directory to save output files
output_directory = '../renewable_energy_data/data_processed'
os.makedirs(output_directory, exist_ok=True)  # This creates the directory if it doesn't exist

# Process and output each dataset
for key, dataset in processed_datasets.items():
    output_path = os.path.join(output_directory, f'{key}_processed.xlsx')
    dataset.to_excel(output_path, index=False)
    print(f"Saved processed data for {key} to {output_path}")



Saved processed data for WF_site1 to ../renewable_energy_data/data_processed/WF_site1_processed.xlsx
Saved processed data for WF_site2 to ../renewable_energy_data/data_processed/WF_site2_processed.xlsx
Saved processed data for WF_site3 to ../renewable_energy_data/data_processed/WF_site3_processed.xlsx
Saved processed data for WF_site4 to ../renewable_energy_data/data_processed/WF_site4_processed.xlsx
Saved processed data for WF_site5 to ../renewable_energy_data/data_processed/WF_site5_processed.xlsx
Saved processed data for WF_site6 to ../renewable_energy_data/data_processed/WF_site6_processed.xlsx
Saved processed data for SS_site1 to ../renewable_energy_data/data_processed/SS_site1_processed.xlsx
Saved processed data for SS_site2 to ../renewable_energy_data/data_processed/SS_site2_processed.xlsx
Saved processed data for SS_site3 to ../renewable_energy_data/data_processed/SS_site3_processed.xlsx
Saved processed data for SS_site4 to ../renewable_energy_data/data_processed/SS_site4_proce